In [36]:
import cv2
import numpy as np
from skimage.util import img_as_ubyte
import time
from skimage import color, feature, io


In [37]:

def show_in_moved_window(win_name, img, x, y):
    """
    Show an image in a window, where the position of the window can be given
    """
    cv2.namedWindow(win_name)
    cv2.moveWindow(win_name, x, y)
    cv2.imshow(win_name, img)


In [38]:
def process_hsv_image(img):
    """
    Simple processing of a color (HSV) image.
    Create a black and white picture where the red objects are white
    """
    hue = img[:, :, 0]
    sat = img[:, :, 1]
    val = img[:, :, 2]

    mask = ((hue < 0.05) | (hue > 0.95)) & (sat > 0.35) & (val > 0.2)
    mask = img_as_ubyte(mask)  

    # Remove noise 
    mask = cv2.medianBlur(mask, 5) # median filter
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


In [39]:
def process_rgb_image(img):
    """
    Segmentation of red structures in the RGB channel
    """
    r_comp = img[:, :, 0]
    g_comp = img[:, :, 1]
    b_comp = img[:, :, 2]
    
    segm = (r_comp > 160) & (r_comp < 180) & (g_comp > 50) & (g_comp < 80) & \
                (b_comp > 50) & (b_comp < 80)
    

    return img_as_ubyte(segm)


In [40]:
def blob_detection(image, new_frame):
    blobs = feature.blob_dog(image, min_sigma = 20, max_sigma = 40, threshold=0.2)
    blobs[:, 2] = blobs[:, 2] * (2 ** 0.5)
    count = 0
    if blobs.size > 0:
        for i, (y, x, r) in enumerate(blobs, start=1):
            if 5 <= r <= 30:
                count += 1
                cv2.circle(new_frame, (int(x), int(y)), int(r), (0, 255, 0), 2)   
                cv2.circle(new_frame, (int(x), int(y)), 2, (0, 255, 0), -1)       
                cv2.putText(new_frame, f"#{i} ({int(x)},{int(y)}) r={int(r)}",
                        (int(x)+6, int(y)-6), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1, cv2.LINE_AA)
    return blobs

In [41]:
def open_camera():
    print("Opening camera")
    url = "http://172.20.10.8:4747/video"
    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print("Cannot open camera")
        raise RuntimeError("Cannot open camera")
    
    print("Starting camera loop")
    # To keep track of frames per second using a high-performance counter
    old_time = time.perf_counter()
    fps = 0
    stop = False
    process_rgb = False
    while not stop:
        ret, new_frame = cap.read()
        if not ret:
            print("Can't receive frame. Exiting ...")
            break

        # Change from OpenCV BGR to scikit image RGB
        new_image = new_frame[:, :, ::-1]
        if process_rgb:
            mask_red = process_rgb_image(new_image)
            
        else:
            new_image = color.rgb2hsv(new_image)
            mask_red = process_hsv_image(new_image)
            blob_detection(mask_red, new_frame)

        # update FPS - but do it slowly to avoid fast changing number
        new_time = time.perf_counter()
        time_dif = new_time - old_time
        old_time = new_time
        fps = fps * 0.95 + 0.05 * 1 / time_dif

        # Put the FPS on the new_frame
        str_out = f"fps: {int(fps)}"
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(new_frame, str_out, (100, 100), font, 1, 255, 1)

        # Display the resulting frame
        show_in_moved_window('Input', new_frame, 0, 10)
        show_in_moved_window('Mask', mask_red, 600, 10)
        if cv2.waitKey(1) == ord('q'):
            stop = True

    print("Stopping image loop")
    cap.release()
    cv2.destroyAllWindows()



In [42]:
open_camera()

Opening camera
Starting camera loop
Can't receive frame. Exiting ...
Stopping image loop


[http @ 0x11393dc0] Stream ends prematurely at 6271353, should be 18446744073709551615
